### Scryfall MKM Price Trend Harvester

This program extracts magic the gathering (mtg) card price data (MKM Trend) at the time the code is run. It can also be used to gather a myriad of other card details that are provided in Sryfall's public API

In [4]:
#import libraries (these will have to be installed on your machine first, using pip)
import requests
import pandas as pd
import datetime

#This one is not necessary; but I work with a small screen so is helpful to utilise margins in jupyter nb
#Switch jupyter to enable multiple outputs per cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#### Use the API to get MKM card prices
See Scryfal REST API Docs at https://scryfall.com/docs/api

I this example I am just using q=cmc:12 to query all cards with converted mana cost (cmc) of 12. But there are loads of other parameters you could use instead--and you can learn about them in the cards section of the API docs at the above link

In [5]:
#get a response object
response=requests.get('https://api.scryfall.com/cards/search?q=cmc:12')
print(response.status_code) #should be 200=ok

200


This response code of 200 indicates that the request was successful (see https://restfulapi.net/http-status-codes/ for more info)

In [6]:
#look at response content
type(response.content) #bytes

#translate the response object's content from bytes to dictionary object so I can manipulate it
results=response.json() # .json() converts from bytes to dictionary
print(type(results)) #type of results in now dict (=dictionary)
print(results) #take a look at results--you can see most of what we want is somewhere in there

bytes

<class 'dict'>
{'object': 'list', 'total_cards': 5, 'has_more': False, 'data': [{'object': 'card', 'id': '7928bb14-7631-4830-a756-26d1ea832ba2', 'oracle_id': 'e80772e2-8623-4094-81a2-70828b2b151c', 'multiverse_ids': [221563], 'mtgo_id': 39587, 'mtgo_foil_id': 39588, 'name': 'Blightsteel Colossus', 'lang': 'en', 'uri': 'https://api.scryfall.com/cards/7928bb14-7631-4830-a756-26d1ea832ba2', 'scryfall_uri': 'https://scryfall.com/card/mbs/99/blightsteel-colossus?utm_source=api', 'layout': 'normal', 'highres_image': True, 'image_uris': {'small': 'https://img.scryfall.com/cards/small/en/mbs/99.jpg?1517813031', 'normal': 'https://img.scryfall.com/cards/normal/en/mbs/99.jpg?1517813031', 'large': 'https://img.scryfall.com/cards/large/en/mbs/99.jpg?1517813031', 'png': 'https://img.scryfall.com/cards/png/en/mbs/99.png?1517813031', 'art_crop': 'https://img.scryfall.com/cards/art_crop/en/mbs/99.jpg?1517813031', 'border_crop': 'https://img.scryfall.com/cards/border_crop/en/mbs/99.jpg?1517813031'}, 'm

In [7]:
#from examining the above I can see that I want the 'data' element from the results dictionary,
#which forms most of the content

#let's look at it
results['data']

[{'object': 'card',
  'id': '7928bb14-7631-4830-a756-26d1ea832ba2',
  'oracle_id': 'e80772e2-8623-4094-81a2-70828b2b151c',
  'multiverse_ids': [221563],
  'mtgo_id': 39587,
  'mtgo_foil_id': 39588,
  'name': 'Blightsteel Colossus',
  'lang': 'en',
  'uri': 'https://api.scryfall.com/cards/7928bb14-7631-4830-a756-26d1ea832ba2',
  'scryfall_uri': 'https://scryfall.com/card/mbs/99/blightsteel-colossus?utm_source=api',
  'layout': 'normal',
  'highres_image': True,
  'image_uris': {'small': 'https://img.scryfall.com/cards/small/en/mbs/99.jpg?1517813031',
   'normal': 'https://img.scryfall.com/cards/normal/en/mbs/99.jpg?1517813031',
   'large': 'https://img.scryfall.com/cards/large/en/mbs/99.jpg?1517813031',
   'png': 'https://img.scryfall.com/cards/png/en/mbs/99.png?1517813031',
   'art_crop': 'https://img.scryfall.com/cards/art_crop/en/mbs/99.jpg?1517813031',
   'border_crop': 'https://img.scryfall.com/cards/border_crop/en/mbs/99.jpg?1517813031'},
  'mana_cost': '{12}',
  'cmc': 12.0,
  't

In [8]:
#use len to see how many items are in the results['data'] for this response
print('results["data"] contains '+str(len(results['data']))+' elements')

results["data"] contains 5 elements


Now that the I can see the card data, the next challenge is to extract the bits I want.. I will try for the 4th element (chosen randomly) and see if I can actually get the elements I need, which are 'card name', 'set', 'proxy for cardkingdom price', and 'cardmarket trend price'... I can get them all as shown below by referencing nested elements of the results['data'] sub-dictionaries. There are loads more available. You can get them by studying the output above and experimenting a bit more

In [9]:
#learn how to extract the data I want
print('name='+results['data'][4]['name'])
print('set_name='+results['data'][4]['set_name'])
print('set='+results['data'][4]['set'])
print('usd='+results['data'][4]['usd'])
print('eur='+str(results['data'][4]['eur']))

name=It That Betrays
set_name=Duel Decks: Zendikar vs. Eldrazi
set=ddp
usd=6.36
eur=2.89


#### Loop It
Last thing to do now is just extract the above elements in a loop--looping through all the elements in the dictionary. You can see that below I am just identifying them for each element in results['data'] (i.e. for each card returned)

Then once identified, I store them to a temp dataframe called df; then finally append that df to my repository dataframe (repo).

Lastly, I just output the repo dataframe to a csv file

In [10]:
#loop through the results and hive the good stuff
repo=pd.DataFrame()
for i in range(0,len(results['data'])):
    card_name=results['data'][i]['name']
    set_name=results['data'][i]['set_name']
    set_tckr=results['data'][i]['set']
    usd=results['data'][i]['usd']
    eur=results['data'][i]['eur']
    df=pd.DataFrame({'card_name':[card_name],
                     'set':set_tckr,
                    'set_name':set_name,
                    'usd':usd,
                    'eur':eur})
    repo=repo.append(df,ignore_index=True)

#preview the dataframe
print(repo)

#store current date (for filename later)
currdate=datetime.datetime.today().strftime('%Y%m%d')
repo.to_csv('cards_'+currdate+'.csv',index=False) #write the df to a csv using the datestamp from above

               card_name  set                          set_name    usd    eur
0   Blightsteel Colossus  mbs                 Mirrodin Besieged  42.70  20.95
1     Enter the Infinite  gtc                         Gatecrash   2.49   1.45
2  Ghalta, Primal Hunger  rix                  Rivals of Ixalan   8.86   6.53
3           Iname as One  sok               Saviors of Kamigawa   0.53   0.55
4        It That Betrays  ddp  Duel Decks: Zendikar vs. Eldrazi   6.36   2.89
